In [1]:
import pandas as pd
import numpy as np
import xlrd
import os

In [2]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('orig_data/mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('orig_data/mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('orig_data/mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('orig_data/mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('orig_data/mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('orig_data/mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('orig_data/mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('orig_data/mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('orig_data/mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('orig_data/mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [3]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [4]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['run_line_close'] = dfnew[0].apply(float)
        df['run_line_odds_close'] = dfnewer.apply(int)
    except KeyError:
        pass

In [5]:
for df in dfList:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.rename({"vh": "visitor_home", "final": "team_run_total", "close":"money_line_close", "open":"money_line_open", "openou":"over_under_line_open", "open_ou":"over_under_line_open", "unnamed:_18":"over_under_odds_open", "unnamed:_19":"over_under_odds_open", "close_ou":"over_under_line_close", "closeou":"over_under_line_close", "unnamed:_20":"over_under_odds_close", "unnamed:_21":"over_under_odds_close"}, axis='columns', inplace=True)    

In [6]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['over_under_line_close'] = df['over_under_line_close'].apply(convt)

In [7]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                totRuns.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                counter += row.team_run_total
            else:
                counter += row.team_run_total
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['total_runs_game'] = totRuns

In [8]:
for df in dfList:
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                RunDif.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                team1 = row.team_run_total
            else:
                team2 = row.team_run_total
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['run_dif_game'] = RunDif

In [9]:
for df in dfList:
    df['date'] = df['year'].astype(str) + '-' + df['date'].astype(str).apply(lambda x: '0' + x if len(x) == 3 else x)
    df['date'] = df['date'].apply(lambda x: x[:7] + '-' + x[7:])

In [10]:
df1 = pd.concat(dfList, sort=True, ignore_index=True)

In [11]:
df1.drop(['year', 'rl', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th'], axis=1, inplace=True)

In [12]:
df1 = df1[df1['visitor_home'] != 'N']

In [13]:
colNew = []
i = 0
for index, row in df1.iterrows():
    if index == 0:
        pass
    elif index % 2 == 0:
        i += 1
    colNew.append(i)
df1['gameno'] = colNew
df1.replace('LOS','LAD',inplace=True)
df1.replace('CUB','CHC',inplace=True)
df1.replace('SDG','SD',inplace=True)
df1.replace('SFO','SF',inplace=True)
df1.replace('KAN','KC',inplace=True)
df1.replace('TAM','TB',inplace=True)

In [14]:
df1 = df1.reindex(columns=['date', 'gameno', 'team', 'visitor_home', 'team_run_total', 'total_runs_game', 'money_line_close', 'money_line_open', 'over_under_line_close', 'over_under_line_open', 'over_under_odds_close', 'over_under_odds_open', 'pitcher', 'rot', 'run_dif_game', 'run_line_close', 'run_line_odds_close'])

In [15]:
df1.replace(['NL', 'nan'], np.nan, inplace=True)

In [16]:
#merge visitor and home rows together
df_v = df1.loc[df1['visitor_home'] == 'V']
df_h = df1.loc[df1['visitor_home'] == 'H']
df_h.columns = ['h_date', 'gameno', 'h_team', 'home', 'h_team_run_total',
                   'h_total_runs_game', 'h_money_line_close', 'h_money_line_open',
                   'under_line_close', 'under_line_open',
                   'under_odds_close', 'under_odds_open', 'h_pitcher', 'h_rot',
                   'h_run_dif_game', 'h_run_line_close', 'h_run_line_odds_close']
df_v.columns = ['date', 'gameno', 'v_team', 'visitor', 'v_team_run_total',
                   'total_runs_game', 'v_money_line_close', 'v_money_line_open',
                   'over_line_close', 'over_line_open',
                   'over_odds_close', 'over_odds_open', 'v_pitcher', 'v_rot',
                   'v_run_dif_game', 'v_run_line_close', 'v_run_line_odds_close']

df3 = pd.merge(df_v, df_h, on='gameno', how='right')

In [17]:
#drop unnecessary columns
df3.drop(['h_total_runs_game','h_date'], axis=1, inplace=True)

In [18]:
#reindex columns
df_final = df3.reindex(columns = ['date', 'gameno', 
             
             'visitor', 'v_team', 'v_team_run_total', 
             'v_money_line_close', 'v_money_line_open', 
             'v_run_line_close', 'v_run_line_odds_close', 
             'v_pitcher', 'v_rot', 'v_run_dif_game',
             
             'home', 'h_team', 'h_team_run_total', 
             'h_money_line_close', 'h_money_line_open', 
             'h_run_line_close', 'h_run_line_odds_close',
             'h_pitcher', 'h_rot', 'h_run_dif_game',
             
             'total_runs_game', 
             'over_line_close', 'over_line_open', 'over_odds_close', 'over_odds_open', 
             'under_line_close', 'under_line_open', 'under_odds_close', 'under_odds_open' 
            ])

In [19]:
df3

,date,gameno,v_team,visitor,v_team_run_total,total_runs_game,v_money_line_close,v_money_line_open,over_line_close,over_line_open,...,h_money_line_open,under_line_close,under_line_open,under_odds_close,under_odds_open,h_pitcher,h_rot,h_run_dif_game,h_run_line_close,h_run_line_odds_close
0,2010-04-04,0,NYY,V,7.0,16.0,107.0,104,9.0,9.0,...,-114,9.0,9.0,-115,-104,JBECKETT-R,932,2.0,NaN,NaN
1,2010-04-05,1,PHI,V,11.0,12.0,-200.0,-200,7.5,7.5,...,175,7.5,7.5,105,-120,JLANNAN-L,902,-10.0,NaN,NaN
2,2010-04-05,2,MIA,V,1.0,8.0,-102.0,105,7.0,7.0,...,-115,7.0,7.0,110,105,JOSANTANA-L,904,6.0,NaN,NaN
3,2010-04-05,3,STL,V,11.0,17.0,-143.0,-155,7.5,7.5,...,140,7.5,7.5,-115,-115,AHARANG-R,906,-5.0,NaN,NaN
4,2010-04-05,4,LAD,V,5.0,16.0,-126.0,-155,8.5,8.5,...,140,8.5,8.5,100,-110,ZDUKE-L,908,6.0,NaN,NaN
5,2010-04-05,5,COL,V,5.0,8.0,-105.0,110,7.5,7.5,...,-125,7.5,7.5,-125,-115,YGALLARDO-R,910,-2.0,NaN,NaN
6,2010-04-05,6,CHC,V,5.0,21.0,110.0,115,8.5,8.5,...,-130,8.5,8.5,-120,-110,DLOWE-R,912,11.0,NaN,NaN
7,2010-04-05,7,SD,V,3.0,9.0,166.0,155,8.0,8.0,...,-175,8.0,8.0,100,-110,DHAREN-R,914,3.0,NaN,NaN
8,2010-04-05,8,SF,V,5.0,7.0,-147.0,-130,7.0,7.5,...,115,7.0,7.5,-110,-125,ROSWALT-R,916,-3.0,NaN,NaN
9,2010-04-05,9,CLE,V,0.0,6.0,127.0,150,9.0,9.0,...,-170,9.0,9.0,-115,-115,MBUEHRLE-L,918,6.0,NaN,NaN


In [20]:
pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', 'dataset.pickle'))
df_final.to_pickle(pickle_path)

In [21]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22326 entries, 0 to 22325
Data columns (total 31 columns):
date                     22326 non-null object
gameno                   22326 non-null int64
visitor                  22326 non-null object
v_team                   22326 non-null object
v_team_run_total         22325 non-null float64
v_money_line_close       22326 non-null float64
v_money_line_open        22326 non-null int64
v_run_line_close         12461 non-null float64
v_run_line_odds_close    12461 non-null float64
v_pitcher                22306 non-null object
v_rot                    22326 non-null int64
v_run_dif_game           22325 non-null float64
home                     22326 non-null object
h_team                   22326 non-null object
h_team_run_total         22325 non-null float64
h_money_line_close       22326 non-null float64
h_money_line_open        22326 non-null int64
h_run_line_close         12461 non-null float64
h_run_line_odds_close    12461 non-null f

In [22]:
df_final['date'] = pd.to_datetime(df_final['date'], infer_datetime_format=True)
test_df = df_final[(df_final['date'] > '2018-01-01') & (df_final['date'] <= '2019-5-1')]

In [23]:
test_df

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,...,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open
19721,2018-03-29,19721,V,CHC,8.0,-200.0,-175,-1.5,-125.0,JLESTER-L,...,-4.0,12.0,8.0,8.0,-120,-105,8.0,8.0,100,-115
19722,2018-03-29,19722,V,STL,4.0,123.0,110,1.5,-185.0,CMARTINEZ-R,...,5.0,13.0,6.5,7.0,-125,-106,6.5,7.0,105,-114
19723,2018-03-29,19723,V,PHI,5.0,-131.0,-125,-1.5,125.0,ANOLA-R,...,3.0,13.0,8.0,8.0,-119,-110,8.0,8.0,-101,-110
19724,2018-03-29,19724,V,MIL,2.0,-131.0,-115,-1.5,130.0,CANDERSON-R,...,-1.0,3.0,8.0,8.0,-115,-110,8.0,8.0,-105,-110
19725,2018-03-29,19725,V,SF,1.0,266.0,260,1.5,125.0,TBLACH-L,...,-1.0,1.0,7.5,7.0,-116,100,7.5,7.0,-104,-120
19726,2018-03-29,19726,V,COL,2.0,-101.0,-115,1.5,-211.0,JGRAY-R,...,6.0,10.0,8.0,9.0,-118,-119,8.0,9.0,-102,-101
19727,2018-03-29,19727,V,MIN,2.0,112.0,110,1.5,-185.0,JODORIZZI-R,...,1.0,5.0,9.5,9.5,-105,-101,9.5,9.5,-115,-119
19728,2018-03-29,19728,V,HOU,4.0,-160.0,-168,-1.5,-110.0,JVERLANDER-R,...,-3.0,5.0,9.0,9.0,-119,-120,9.0,9.0,-101,100
19729,2018-03-29,19729,V,NYY,6.0,-171.0,-155,-1.5,-109.0,LSEVERINO-R,...,-5.0,7.0,8.5,8.5,-101,-105,8.5,8.5,-119,-115
19730,2018-03-29,19730,V,BOS,4.0,-173.0,-175,-1.5,-114.0,CSALE-L,...,2.0,10.0,6.5,7.0,-119,-119,6.5,7.0,-101,-101


In [24]:
1/120/100

8.333333333333333e-05